In [1]:
import sys
# import pyccl as ccl
import pickle
import camb
sys.path.insert(0,'../skylens/')
# from importlib import reload

In [2]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
c=LocalCluster(n_workers=1,processes=False,memory_limit='25gb',threads_per_worker=14,memory_spill_fraction=.99,
               memory_monitor_interval='2000ms')
client=Client(c)

In [3]:
from skylens import *
from survey_utils import *

In [4]:
#only for python3
import importlib
reload=importlib.reload

In [5]:
zs=lsst_source_tomo_bins()
corr_ll=('shear', 'shear')

ns0:  27.0


In [6]:
wigner_files={}
# wigner_files[0]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_0_reorder.zarr'
# wigner_files[2]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_2_reorder.zarr'

wigner_files[0]= '/home/deep/repos/cosmic_shear/temp/dask_wig3j_l6500_w2100_0_reorder.zarr'
wigner_files[2]= '/home/deep/repos/cosmic_shear/temp/dask_wig3j_l3500_w2100_2_reorder.zarr'


# Power Spectrum covariance

In [7]:
#setup parameters
lmax_cl=200
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=20
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell

do_cov=True
bin_cl=True

SSV_cov=True
tidal_SSV_cov=False
Tri_cov=True

bin_xi=True
theta_bins=np.logspace(np.log10(1./60),1,20)

##  Test with 1 redshift bin

In [8]:
store_win=True
window_lmax=200

In [9]:
use_window=True
do_cov=True

In [10]:
nside=128

In [11]:
z0=1 #1087
zs_bin1=source_tomo_bins(zp=[z0],p_zp=np.array([1]),ns=30,use_window=use_window,nside=nside)

indxs:  None None
Win gen: step size 384 True
Window done
[1.] [1.00000000e-04 1.09761018e-04 1.20474810e-04 1.32234377e-04
 1.45141798e-04 1.59309115e-04 1.74859306e-04 1.91927353e-04
 2.10661416e-04 2.31224114e-04 2.53793941e-04 2.78566812e-04
 3.05757768e-04 3.35602838e-04 3.68361090e-04 4.04316881e-04
 4.43782323e-04 4.87099994e-04 5.34645911e-04 5.86832792e-04
 6.44113645e-04 7.06985691e-04 7.75994689e-04 8.51739668e-04
 9.34878127e-04 1.02613175e-03 1.12629265e-03 1.23623027e-03
 1.35689893e-03 1.48934607e-03 1.63472140e-03 1.79428685e-03
 1.96942750e-03 2.16166367e-03 2.37266404e-03 2.60426020e-03
 2.85846250e-03 3.13747753e-03 3.44372726e-03 3.77987009e-03
 4.14882387e-03 4.55379130e-03 4.99828768e-03 5.48617142e-03
 6.02167758e-03 6.60945459e-03 7.25460462e-03 7.96272786e-03
 8.73997113e-03 9.59308126e-03 1.05294636e-02 1.15572464e-02
 1.26853513e-02 1.39235707e-02 1.52826528e-02 1.67743953e-02
 1.84117470e-02 2.02089208e-02 2.21815172e-02 2.43466590e-02
 2.67231407e-02 2.9331

../skylens/cov_utils.py:58: RuntimeWarning: invalid value encountered in true_divide
  Win0=2*jn(1,l_th)/l_th
../skylens/angular_power_spectra.py:26: RuntimeWarning: divide by zero encountered in true_divide
  self.cl_f=(l+0.5)**2/(l*(l+1.)) # cl correction from Kilbinger+ 2017
/usr/local/lib/python3.8/dist-packages/healpy/sphtfunc.py:396: FutureChangeWarning: The order of the input cl's will change in a future release.
Use new=True keyword to start using the new order.
See documentation of healpy.synalm.
  warnings.warn(


In [12]:
use_binned_l=False

In [13]:
a={'a':1,'b':2}

In [14]:
kk=list(a.keys())
ii=len(kk)

In [15]:
[(kk[i],kk[j]) for i in np.arange(ii) for j in np.arange(i,ii)]

[('a', 'a'), ('a', 'b'), ('b', 'b')]

In [16]:
import skylens
reload(skylens)
from skylens import *

In [17]:
store_win=True

In [18]:
#use all ell
kappa0=Skylens(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       use_binned_l=use_binned_l,wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax
                                       )

In [19]:
%time cl0G=kappa0.cl_tomo() 

In [20]:
kappa0.l.shape

(198,)

In [21]:
%time cl0=cl0G['stack'].compute()

In [22]:
cl0

{'cov': array([[1.49532967e-17, 3.58365481e-18, 3.13155028e-19, 7.38686346e-20,
         3.52719132e-20, 2.23027350e-20, 1.21662330e-20, 8.84425084e-21,
         6.85693823e-21, 5.30347023e-21, 4.24332822e-21, 3.42374465e-21,
         2.78009928e-21, 2.26851187e-21, 1.85209032e-21, 1.53197893e-21,
         1.27423374e-21, 1.06783085e-21, 8.91616242e-22],
        [3.58365481e-18, 7.96527319e-18, 1.82136484e-18, 1.65194304e-19,
         5.14335577e-20, 2.40280313e-20, 1.29356188e-20, 8.74926601e-21,
         6.39384596e-21, 5.00452659e-21, 3.92730426e-21, 3.15411562e-21,
         2.56258825e-21, 2.08211135e-21, 1.69777477e-21, 1.39883280e-21,
         1.16228718e-21, 9.72161543e-22, 8.09777275e-22],
        [3.13155028e-19, 1.82136484e-18, 7.77499341e-18, 1.41013287e-18,
         1.14822020e-19, 3.37763330e-20, 1.45393776e-20, 8.47923052e-21,
         6.04154989e-21, 4.59886571e-21, 3.59414164e-21, 2.86774122e-21,
         2.33058849e-21, 1.88242673e-21, 1.53163725e-21, 1.25757115e-21,
 

In [23]:
do_xi=True
bin_xi=True
do_cov=False
bin_cl=True
th_min=2.5/60
th_max=250./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins*40)
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

In [24]:
# import wigner_transform 
# reload(wigner_transform)
# from wigner_transform import *

import binning
reload(binning)
from binning import *

In [25]:
th

array([0.04166667, 0.04190751, 0.04214975, 0.04239339, 0.04263844,
       0.0428849 , 0.04313279, 0.04338211, 0.04363287, 0.04388508,
       0.04413875, 0.04439389, 0.0446505 , 0.04490859, 0.04516818,
       0.04542926, 0.04569186, 0.04595597, 0.04622161, 0.04648879,
       0.04675751, 0.04702778, 0.04729961, 0.04757302, 0.04784801,
       0.04812458, 0.04840276, 0.04868254, 0.04896394, 0.04924697,
       0.04953163, 0.04981794, 0.0501059 , 0.05039553, 0.05068683,
       0.05097982, 0.0512745 , 0.05157088, 0.05186897, 0.05216879,
       0.05247034, 0.05277364, 0.05307869, 0.0533855 , 0.05369408,
       0.05400445, 0.05431661, 0.05463058, 0.05494636, 0.05526397,
       0.05558341, 0.0559047 , 0.05622785, 0.05655286, 0.05687976,
       0.05720854, 0.05753922, 0.05787182, 0.05820633, 0.05854278,
       0.05888118, 0.05922153, 0.05956385, 0.05990815, 0.06025443,
       0.06060272, 0.06095303, 0.06130535, 0.06165972, 0.06201613,
       0.0623746 , 0.06273515, 0.06309777, 0.0634625 , 0.06382

In [26]:
#Hankel Transform setup
WT_kwargs={'l':l0,'theta':th,'s1_s2':[(2,2),(2,-2),(0,0)]}
WT=wigner_transform(**WT_kwargs)

In [27]:
WT.wig_d.keys()

dict_keys([(2, 2), (2, -2), (0, 0)])

In [28]:
import skylens
reload(skylens)
from skylens import *

In [29]:
use_binned_theta=True

In [30]:
kappa_xi0=Skylens(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax,
                                      do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                                      use_binned_l=False,use_binned_theta=False,
                                      nz_PS=10
                                       )

In [31]:
%time xiHG0=kappa_xi0.xi_tomo()
%time xiH0=xiHG0['stack'].compute()

../skylens/skylens.py:633: RuntimeWarning: invalid value encountered in true_divide
  xi_b/=(Win['cl'][corr][indxs]['xi_b'])


In [32]:
kappa_xib=Skylens(zs_bins=zs_bin1,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, zg_bins=None,
                                       use_window=use_window,Tri_cov=Tri_cov,
                                       wigner_files=wigner_files,
                                       SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
                                       store_win=store_win,window_lmax=window_lmax,
                                      do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                                      use_binned_l=use_binned_l,use_binned_theta=use_binned_theta
                                       )

In [33]:
%time xiHG_b=kappa_xib.xi_tomo()

In [34]:
%time xiH_b=xiHG_b['stack'].compute()

distributed.worker - WARNING -  Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7f5d8373ba60>, <bound method Skylens.calc_pseudo_cl of <skylens.Skylens object at 0x7f5d5c242c10>>, [array([4.18096709e-08, 4.04226424e-08, 3.79676924e-08, 3.62456244e-08,
       3.36864394e-08, 3.04718981e-08, 2.75732203e-08, 2.44211407e-08,
       2.12755975e-08, 1.87156681e-08, 1.60121554e-08, 1.39091923e-08,
       1.18682812e-08, 9.99784060e-09, 8.44144225e-09, 6.97693947e-09,
       5.74687230e-09, 4.70601010e-09, 3.89613523e-09])], (<class 'dict'>, [['Win', (<class 'dict'>, [['cl', (<class 'dict'>, [[(<class 'tuple'>, ['shear', 'shear']), (<class 'dict'>, [[(<class 'tuple'>, [0, 0]), (<class 'dict'>, [['corr', (<class 'tuple'>, ['shear', 'shear'])], ['indxs', (<class 'tuple'>, [0, 0])], [12, (<class 'dict'>, [['cl', array([1.13095832e+00, 5.54039065e-01, 8.85339706e-02, 1.41170149e-03,
       1.96974456e-02, 4.31695815e-03, 1.49864067e-03, 4.37287519e-03,
       3.37698473e-0

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 198 is different from 19)

In [35]:
kappa_xi0.WT.wig_d.keys(),WT.wig_d.keys()

(dict_keys([(2, 2), (2, -2), (0, 0)]), dict_keys([(2, 2), (2, -2), (0, 0)]))

In [36]:
%time xiHG0=kappa_xi0.xi_tomo()
%time xiH0=xiHG0['stack'].compute()

In [37]:
xiH0['xi']

array([           nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan, 1.06149426e-05, 5.45404466e-06, 2.73327220e-06,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan, 1.64542830e-05, 1.10299289e-05, 1.07098848e-05])

In [38]:
xiH_b['xi']

NameError: name 'xiH_b' is not defined